In [1]:
import pyreadr
import pandas as pd

from dash import Dash, html, dcc, callback, Output, Input, dash_table
import dash_bootstrap_components as dbc

import plotly.express as px

I import the dcc module (Dash Core Components), which includes a Graph component, used to render interactive graphs. Also import the plotly.express library to build the interactive graphs.
Using the px I build the chart and assign it to the figure property of dcc.Graph. This displays the chart in the app.

Add radio buttons to the app layout. 

Then, build the **callback** to create the interaction between the buttons and the chart. To work with the callback, import the callback module and the two arguments commonly used within the callback: Output and Input.

Both the RadioItems and the Graph components were given id names: used by the callback to identify the components.

The inputs and outputs of our app are the properties of a particular component. For example, input is the value property of the component that has the ID "controls-and-radio-item". Output is the figure property of the component with the ID "controls-and-graph", which is currently an empty dictionary (empty graph).

The callback function's argument col_chosen refers to the component property of the input. We build the chart inside the callback function. Every time the user selects a new radio item, the figure is rebuilt / updated. Return the graph at the end of the function. This assigns it to the figure property of the dcc.Graph, thus displaying the figure in the app.

In [2]:
def read_res(depname, cmrname, path='/Users/Serena/Desktop/Bath_visit/2.NETWORK/Results/'):
    res = pyreadr.read_r(f'{path}{depname}_{cmrname}.RData')
    dep_names = list(res['dep_names']['dep_names'])
    cmr_names = list(res['cmr_names']['cmr_names'])
    fit_meas = res['fit_meas'].T
    unst_est = res['unst_est']
    stad_est = res['stad_est']
    return(dep_names, cmr_names, fit_meas, unst_est, stad_est)

# deps, cmrs, fm, ue, se = read_res('sDEP','FMI')

In [3]:
def make_table(depname, cmrname):
    deps, cmrs, _,_,_ = read_res(depname,cmrname)
    times = pd.DataFrame([['Depression']+[x.split('_')[-1] for x in deps], 
               ['Cardio-metabolic risk']+[x.split('_')[-1] for x in cmrs]])
    # .rename(columns = {0:'Marker'}, index={0:'dep',1:'cmr'})
    return(times)

In [4]:
def make_net(depname, cmrname, width=1000):
    
    deps, _, _, ue, se = read_res(depname, cmrname)
    
    ls1 = list(round(ue.loc[(ue.lhs=='eta_dep')&(ue.op=='=~'), 'est'], 2))
    ls2 = list(round(ue.loc[(ue.lhs=='eta_cmr')&(ue.op=='=~'), 'est'], 2))

    # ar_dep = list(round(se.loc[se.label.str.contains('^AR_dep')].iloc[::-1]['est.std'], 2))
    # ar_cmr = list(round(se.loc[se.label.str.contains('^AR_cmr')].iloc[::-1]['est.std'], 2))
    ar_dep = list(round(ue.loc[ue.label.str.contains('^AR_dep')].iloc[::-1]['est'], 2))
    ar_cmr = list(round(ue.loc[ue.label.str.contains('^AR_cmr')].iloc[::-1]['est'], 2))
    
    #cl_dep = list(round(se.loc[se.label.str.contains('^CL_dep')].iloc[::-1]['est.std'], 2))
    #cl_cmr = list(round(se.loc[se.label.str.contains('^CL_cmr')].iloc[::-1]['est.std'], 2))
    cl_dep = list(round(ue.loc[ue.label.str.contains('^CL_dep')].iloc[::-1]['est'], 2))
    cl_cmr = list(round(ue.loc[ue.label.str.contains('^CL_cmr')].iloc[::-1]['est'], 2))
    
    
    nt = len(deps) # Number of timepoints
    
    e = []
    # Eta factors nodes
    e.append({'data': {'id':'eta1', 'label':'Eta'}, 'position':{'x':width/2,'y':30} })
    e.append({'data': {'id':'eta2', 'label':'Eta'}, 'position':{'x':width/2,'y':550} })
    for i in range(1,nt+1): 
        # The nodes elements
        e.append({'data': {'id':'dep'+str(i), 'label':'Depression '+str(i)}, 
                  'position':{'x':((width/nt)*i)-(width/nt)/2,'y':180} })
        e.append({'data': {'id':'cmr'+str(i), 'label':'Cardiometa '+str(i)}, 
                  'position':{'x':((width/nt)*i)-(width/nt)/2,'y':400} })
        # The edge element
        e.append({'data': {'source':'eta1', 'target':'dep'+str(i), 'weight': ls1[i-1], 'label': 'lambda'+str(i)}})
        e.append({'data': {'source':'eta2', 'target':'cmr'+str(i), 'weight': ls2[i-1], 'label': 'lambda'+str(i)}})
        if i < nt:
            e.append({'data': {'source':'dep'+str(i), 'target':'dep'+str(i+1), 'weight': ar_dep[i-1], 'label': 'AR'+str(i)}})
            e.append({'data': {'source':'cmr'+str(i), 'target':'cmr'+str(i+1), 'weight': ar_cmr[i-1], 'label': 'AR'+str(i)}})
            e.append({'data': {'source':'dep'+str(i), 'target':'cmr'+str(i+1), 'weight': cl_dep[i-1], 'label': 'CL'+str(i)}})
            e.append({'data': {'source':'cmr'+str(i), 'target':'dep'+str(i+1), 'weight': cl_cmr[i-1], 'label': 'CL'+str(i)}})
        
    return(e)

In [6]:
import dash_cytoscape as cyto

app = Dash(__name__, external_stylesheets=[dbc.themes.SIMPLEX])

app.layout = html.Div([
    # Title
    html.H1(children='Cross-lag panel model', style={'textAlign':'center'}),
    html.Hr(),
    # Input 
    dbc.Row([dbc.Col([html.H5(children='Depression score', style={'textAlign':'left'}),
                     dcc.RadioItems(options=[{'label': 'Self-reported', 'value': 'sDEP'},
                                             {'label': 'Maternal report', 'value': 'mDEP'}],
                                    value='sDEP', id='dep-selection')]), 
             dbc.Col([html.H5(children='Cardio-metabolic marker', style={'textAlign':'left'}),
                     dcc.Dropdown(options=[{'label': 'Fat mass index (FMI)', 'value': 'FMI'},
                                           {'label': 'Body mass index (BMI)', 'value': 'BMI'},
                                           {'label': 'Total fat mass', 'value': 'total_fatmass'},
                                           {'label': 'Waist circumference', 'value': 'waist_circ'}],
                                  value='FMI', id='cmr-selection')]) ]),
    html.Hr(),
    # Table
    dash_table.DataTable(id='time-table', data = make_table('sDEP', 'FMI').to_dict('records'), page_size=10),
    # Network
    cyto.Cytoscape(id='cyto-graph',
        layout={'name': 'preset', 'fit':False},
        style={'width': '100%', 'height': '1000px'},
        minZoom=0.8, maxZoom=1, # reduce the range of user zooming 
        elements = make_net('sDEP', 'FMI'), 
        stylesheet=[
            # Annotate edge weight
            {'selector':'edge', 'style':{'width': 3, 'target-arrow-shape':'vee', 'curve-style':'straight'} },
            # shape & color nodes
            {'selector':'[label != "Eta"]','style': {'shape': 'rectangle', 'height':25,
                      'background-color': 'white','border-color': 'black','border-width':2}},
            {'selector':'[label = "Eta"]','style': {'shape': 'round', 'height':20,
                      'background-color': 'lightblue','border-color': 'blue','border-width':2}},
            # Color lines
            {'selector':'[label *= "la"]', 'style':{'line-color':'blue', 'target-arrow-color':'blue', 'arrow-scale':1.5,
                                                    'label':'data(weight)', 'font-size':15,
                                                    'text-background-color':'blue', 'text-background-opacity':.5 }},
            {'selector':'[label *= "AR"]', 'style':{'line-color':'red', 'target-arrow-color':'red', 'arrow-scale':1.5,
                                                    'label':'data(weight)', 'font-size':20,'font-weight':'bold',
                                                    'text-background-color':'red', 'text-background-opacity':.5 }},
            {'selector':'[label *= "CL"]', 'style':{'line-color':'green','target-arrow-color':'green', 'arrow-scale':1.5,
                                                    'source-label':'data(weight)','font-size':20,'font-weight':'bold',
                                                    'source-text-offset': 50, # 'source-endpoint':['-10%','0%']
                                                    'text-background-color':'green', 'text-background-opacity':.5 }} ] )    
])

# Add controls to build the interaction
@callback(
    Output('cyto-graph', 'elements'),
    Input('dep-selection', 'value'),
    Input('cmr-selection', 'value')
)
def update_graph(dep_selection, cmr_selection):
    return make_net(dep_selection, cmr_selection)

@callback(
    Output('time-table', 'data'),
    Input('dep-selection', 'value'),
    Input('cmr-selection', 'value')
)
def update_table(dep_selection, cmr_selection):
    return make_table(dep_selection, cmr_selection).to_dict('records')

if __name__ == '__main__':
    app.run(jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/
